## 1. Create Anyscale Resources with Terraform

Let's set up the necessary Terraform variables and apply the configuration:

### Creating Anyscale Resources

Steps for deploying Anyscale resources via Terraform:

* Review and modify [variables.tf](variables.tf) with your configurations, like `eks_cluster_name`
* (Optional) Create a `terraform.tfvars` file to override any defaults
* View [main.tf](main.tf) to see how the resources are created

In [ ]:
"""
Set the global variables for the deployment, which are same as the variables.tf file.
"""

EKS_CLUSTER_NAME = "anyscale-eks-private"  # Replace with your actual EKS cluster name
AWS_REGION = "us-west-2"  # Replace with your actual AWS region
ANYSCALE_CLOUD_NAME = "anyscale-cloud-eks-private" # Replace with your actual Anyscale cloud name
ANYSCALE_S3_BUCKET_NAME = EKS_CLUSTER_NAME + "-" + AWS_REGION # You can leave it empty for now, you can get it from the output of the terraform apply command, then you can populate it here.

In [ ]:
# Run Terraform commands

# Initialize Terraform
!terraform init

# Preview the changes
!terraform plan

# Apply the changes, (this may take 10-15 minutes)
!terraform apply -auto-approve

<details>
<summary>Sometimes, your AWS account may have too many existing resources consuming capacity, causing you to hit service limits. In such cases, it's a good idea to release unused resources—such as unassociated Elastic IPs (EIPs) or idle NAT gateways.
</summary>
During you installing, you may need to release the unused EIPs and deleted unused NAT.

To find the unattached EIPs:
```bash
aws ec2 describe-addresses | jq '.Addresses[] | select(.InstanceId == null and .NetworkInterfaceId == null)'
```
You will see a list of unattached EIPs.

To release one:

```bash
aws ec2 release-address --allocation-id eipalloc-xxxxxx
```

To find the unused NAT in your region, for example us-west-2:
```bash
aws ec2 describe-nat-gateways --region us-west-2 --filter "Name=state,Values=available" | jq '.NatGateways | length'
```

To delete one:

```bash
aws ec2 delete-nat-gateway --nat-gateway-id nat-xxxxxxxx
```

To identify VPCs that look like they might be safe to delete (test/development ones):
```bash
aws ec2 describe-vpcs --query 'Vpcs[*].[VpcId,Tags[?Key==`Name`].Value|[0]]' --output table | grep -E "(test|temp|dev|scratch|derp|floral|scrumptious)"
```

to delete one:
```bash
aws ec2 delete-vpc --vpc-id vpc-0f8bb12ddf9a451e9  # You can delete more
```

You may need to delete more depends on your scenario.

</detail>

<div class="alert alert-block alert-info">
<b>Take a note to the output of terraform apply! </b>You will need it when you register the Anyscale cloud to your cloud provider.
</div>

<details>
<summary>Sample output</summary>
```
Outputs:

anyscale_registration_command = <<EOT
anyscale cloud register \
        --name <anyscale_cloud_name> \
        --region xxxxxxx \
        --provider aws \
        --compute-stack k8s \
        --kubernetes-zones us-west-2a,us-west-2b \
        --s3-bucket-id xxxxxxx \
        --anyscale-operator-iam-identity arn:aws:iam::xxxxxx:role/default-eks-node-group-xxxxxxxxxxxx
EOT
aws_region = "xxxxxxxx"
eks_cluster_name = "xxxxxxx"
helm_upgrade_command = <<EOT
helm upgrade anyscale-operator anyscale/anyscale-operator \
        --set-string cloudDeploymentId=<cloud-deployment-id> \
        --set-string cloudProvider=aws \
        --set-string region=us-west-2 \
        --set-string workloadServiceAccountName=anyscale-operator \
        --namespace anyscale-operator \
        --create-namespace \
        -i
EOT
```
</details>